# Проект по анализу данных

---

**Тема проекта:** анализ восприятия нейросетей на примере контента в YouTube.

**Исследовательский вопрос**: каким образом пользователи ютуб регируют на контент связанный с  нейросетями?

**Задачи:**
1. Просмотреть связана ли вовлеченность в контент (видео) с категорией видео (первая переменная будет создана на основании переменных из базы; вторая переменная взята из базы в исходном виде).
2. Выяснить, связана ли вовлеченность в контент (видео) с годом выпуска видео (обе переменные будут созданы на основании переменных в базе).


In [ ]:
# № 0. устнановим и импортируем нужные нам  модули/библиотеки
!pip install randan
#!pip install plotly

import googleapiclient.discovery as api # установленный модуль для применения API (только для неавторизованных)
import pandas
import numpy
import math
from randan.descriptive_statistics import ScaleStatistics # количественные переменные
from randan.descriptive_statistics import NominalStatistics # номинальные переменные
from randan.bivariate_association import Correlation # корреляция
from randan.bivariate_association import Crosstab # Хи-квадрат
import plotly.express as px
import random, requests, time # генерация случайного числа, обращения по интересующему URL-адресу и "засыпания" исполнения соответственно

In [18]:
# № 1 авторизируемся по ключу

# добавим ключ для авторизации

API_KEY = "тут ваш ключ апи"
# указали название сервиса
api_service_name = "youtube"

# версия API
api_version = "v3"

# все необходимые данные для авторизации
youtube = api.build(api_service_name, api_version, developerKey = API_KEY)

In [19]:
# № 2 формирование запроса

# запрос
q = "нейросеть"

# источник
type = "video"

In [ ]:
# № 3 подача запроса
request = youtube.search().list(
    part = "snippet"             # snippet отвечает за выдачу набора параметров видео и каналов
    , maxResults = 50
    , q = q
    , type = type)

# сохраняем все собранные данные по запросу в объект response
response = request.execute()

#  это датафрейм, в котором будет сохранена вся информация о первых 50 видео
df_ne = pandas.json_normalize(response["items"])

df_ne

In [21]:
# № 4 смотрим количество существующих видео на момент подачи запроса
display(response["pageInfo"]["totalResults"])

341486

In [22]:
# № 5 обозначение переменной для тех видео, что ищутся по другим сортировкам
order_list = ["date", "rating", "relevance", "title", "videoCount", "viewCount"]
order_list

['date', 'rating', 'relevance', 'title', 'videoCount', 'viewCount']

In [ ]:
#  6  запроса с использованием цикла для сбора данных со всех страниц с выдачей и аргумента order для увеличения выдачи
i = 0

# lанный цикл будет работать до тех пор, пока не пройдется по всем значениям аргумента order
for order in order_list:

    if len(df_ne.drop_duplicates("id.videoId")) < response["pageInfo"]["totalResults"]:
        request = youtube.search().list(
            part = "snippet"
            , maxResults = 50
            , q = q
            , type = type
            , order = order)

        response = request.execute()

        print(f'Итерация №{i}, "order" {order}, "items" {len(response["items"])}')
        i += 1

        df = pandas.pandas.json_normalize(response["items"]) # датафрейм, в котором обработанные данные
        df_ne = pandas.concat([df, df_ne]) # датафрейм, в котором новые данные (df) с каждой страницы (итерации)

        # данный цикл будет работать до тех пор, пока на текущей странице указан токен следующей страницы (nextPageToken)
        while ("nextPageToken" in response.keys())\
        & (len(df_ne.drop_duplicates("id.videoId")) < response["pageInfo"]["totalResults"])\
        & (len(response["items"]) > 0):
              request = youtube.search().list(
                  part = "snippet"
                  , maxResults = 50
                  , q = q
                  , type = type
                  , pageToken = response["nextPageToken"]
                  , order = order)

              response = request.execute()

              print(f'Итерация №{i}, "order" {order}, "items" {len(response["items"])}')
              i += 1
              df_additional = pandas.json_normalize(response["items"])
              df_ne = pandas.concat([df_ne, df_additional])

df_ne

3583 видео (на момент запуска скрипта) получилось по запросу "нейросеть".

In [ ]:
# № 7  удаляем дубликаты
df_ne = df_ne.drop_duplicates("id.videoId")
display(df_ne)

После чистки у нас осталось 935 видео, соответствующих нашему запросу.

In [ ]:
# № 8 достали из таблицы с видео их id
videoId_list = df_ne['id.videoId'].to_list()
videoId_list

In [ ]:
# № 9  делаем все тоже самое, что и раньше только с подачей id
request = youtube.videos().list(
    part = ["snippet"              # тут хранятся: дата публикации видео, id канала, название видео, описание видео, tags, categoryId, название канала
            , "contentDetails"     # тут хранится длительность
            , "statistics"         # тут хранятся: количество просмотров, лайков, комментариев
            , "topicDetails"]      # тут хранятся: категории видео
    , id = videoId_list[0:50]
    , maxResults = 50)

response = request.execute()
video_ne = pandas.json_normalize(response['items'])
video_ne

In [ ]:
# № 10
# цикл для прохода по всем id видео из списка videoId_list
for i in range(50, len(videoId_list)+1, 50):
    request = youtube.videos().list(
        part = ["snippet"
                , "contentDetails"
                , "statistics"
                , "topicDetails"]
        , id = videoId_list[i:i+50]
        , maxResults = 50)
    response = request.execute()

    print(f"Итерация №{i}")

    df_additional = pandas.json_normalize(response["items"]) # датафрейм, в котором будут сохранены собранные характеристики каналов с каждой страницы
    video_ne = pandas.concat([video_ne, df_additional])

video_ne

In [28]:
# № 11 удаляем дубликаты из столбца snippet.categoryId
unique_categories = video_ne["snippet.categoryId"].drop_duplicates().tolist()
print(unique_categories)
print(f"Число уникальных категорий в базе: {len(unique_categories)}")

['22', '24', '27', '23', '25', '28', '26', '10', '1', '20', '29', '2', '17', '19', '15']
Число уникальных категорий в базе: 15


In [29]:
# № 12 подаем запрос
request = youtube.videoCategories().list(
    part = "snippet"
    , id = unique_categories)

response = request.execute()

In [30]:
# № 13 работа с id категориями
category_names_df = pandas.json_normalize(response["items"])
category_names_df

,kind,etag,id,snippet.title,snippet.assignable,snippet.channelId
0,youtube#videoCategory,grPOPYEUUZN3ltuDUGEWlrTR90U,1,Film & Animation,True,UCBR8-60-B28hp2BmDPdntcQ
1,youtube#videoCategory,Q0xgUf8BFM8rW3W0R9wNq809xyA,2,Autos & Vehicles,True,UCBR8-60-B28hp2BmDPdntcQ
2,youtube#videoCategory,qnpwjh5QlWM5hrnZCvHisquztC4,10,Music,True,UCBR8-60-B28hp2BmDPdntcQ
3,youtube#videoCategory,HyFIixS5BZaoBdkQdLzPdoXWipg,15,Pets & Animals,True,UCBR8-60-B28hp2BmDPdntcQ
4,youtube#videoCategory,PNU8SwXhjsF90fmkilVohofOi4I,17,Sports,True,UCBR8-60-B28hp2BmDPdntcQ
5,youtube#videoCategory,ANnLQyzEA_9m3bMyJXMhKTCOiyg,19,Travel & Events,True,UCBR8-60-B28hp2BmDPdntcQ
6,youtube#videoCategory,0Hh6gbZ9zWjnV3sfdZjKB5LQr6E,20,Gaming,True,UCBR8-60-B28hp2BmDPdntcQ
7,youtube#videoCategory,cHDaaqPDZsJT1FPr1-MwtyIhR28,22,People & Blogs,True,UCBR8-60-B28hp2BmDPdntcQ
8,youtube#videoCategory,3Uz364xBbKY50a2s0XQlv-gXJds,23,Comedy,True,UCBR8-60-B28hp2BmDPdntcQ
9,youtube#videoCategory,0srcLUqQzO7-NGLF7QnhdVzJQmY,24,Entertainment,True,UCBR8-60-B28hp2BmDPdntcQ


In [31]:
category_names_df["id"]  # dtype: object меняем во всем столбце на числа
category_names_df["id"] = category_names_df["id"].astype("int")

In [32]:
category_names_df["id"]

0      1
1      2
2     10
3     15
4     17
5     19
6     20
7     22
8     23
9     24
10    25
11    26
12    27
13    28
14    29
Name: id, dtype: int64

In [33]:
# №14 замена сквозной нумерации в таблице с id на номера самих id
category_names_df.index = category_names_df["id"].to_list()
category_names_df

,kind,etag,id,snippet.title,snippet.assignable,snippet.channelId
1,youtube#videoCategory,grPOPYEUUZN3ltuDUGEWlrTR90U,1,Film & Animation,True,UCBR8-60-B28hp2BmDPdntcQ
2,youtube#videoCategory,Q0xgUf8BFM8rW3W0R9wNq809xyA,2,Autos & Vehicles,True,UCBR8-60-B28hp2BmDPdntcQ
10,youtube#videoCategory,qnpwjh5QlWM5hrnZCvHisquztC4,10,Music,True,UCBR8-60-B28hp2BmDPdntcQ
15,youtube#videoCategory,HyFIixS5BZaoBdkQdLzPdoXWipg,15,Pets & Animals,True,UCBR8-60-B28hp2BmDPdntcQ
17,youtube#videoCategory,PNU8SwXhjsF90fmkilVohofOi4I,17,Sports,True,UCBR8-60-B28hp2BmDPdntcQ
19,youtube#videoCategory,ANnLQyzEA_9m3bMyJXMhKTCOiyg,19,Travel & Events,True,UCBR8-60-B28hp2BmDPdntcQ
20,youtube#videoCategory,0Hh6gbZ9zWjnV3sfdZjKB5LQr6E,20,Gaming,True,UCBR8-60-B28hp2BmDPdntcQ
22,youtube#videoCategory,cHDaaqPDZsJT1FPr1-MwtyIhR28,22,People & Blogs,True,UCBR8-60-B28hp2BmDPdntcQ
23,youtube#videoCategory,3Uz364xBbKY50a2s0XQlv-gXJds,23,Comedy,True,UCBR8-60-B28hp2BmDPdntcQ
24,youtube#videoCategory,0srcLUqQzO7-NGLF7QnhdVzJQmY,24,Entertainment,True,UCBR8-60-B28hp2BmDPdntcQ


In [ ]:
# № 15 столбец новый куда поместим категории видео
video_ne["categoryName"] = 0
video_ne

In [ ]:
video_ne["snippet.categoryId"] = video_ne["snippet.categoryId"].astype("int")
video_ne.loc[video_ne["snippet.categoryId"] == 1, "categoryName"] = category_names_df["snippet.title"][1]
video_ne[video_ne["snippet.categoryId"] == 1]

In [ ]:
# цикл для того, чтобы перезаписать расшифровки всех категорий id
for i in category_names_df.index:
    video_ne.loc[video_ne["snippet.categoryId"] == i, "categoryName"] = category_names_df["snippet.title"][i]
video_ne

In [ ]:
# № 16  сквозной индекс для итоговой таблицы
video_ne.index = range(1, len(video_ne)+1)
display(video_ne)

In [38]:
# № 17 теперь наша задача собрать дизлайки для создания переменной вовлеченность, через собственно-ручно созданную функцию  dislikes()
userAgent = ["Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36 RuxitSynthetic/1.0 v3653660536 t3060034875834570496 ath318f403d altpub cvcv=2 smf=0"
             , "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36 RuxitSynthetic/1.0 v7638580655095651333 t3369586550842097326 ath1fb31b7a altpriv cvcv=2 smf=0"
             , "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36 RuxitSynthetic/1.0 v4022270135305624536 t210370085878917362 ath259cea6f altpriv cvcv=2 smf=0"
             , "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36 RuxitSynthetic/1.0 v1605625219501687423 t5534040873462942519 ath5ee645e0 altpriv cvcv=2 smf=0"
             , "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36 RuxitSynthetic/1.0 v9045054735720176711 t8055053424576152628 ath4b3726d5 altpriv cvcv=2 smf=0"
             , "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36 RuxitSynthetic/1.0 v4774994596211760778 t5534040873462942519 ath5ee645e0 altpriv cvcv=2 smf=0"
             , "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36 RuxitSynthetic/1.0 v1418982706 t79976092658796286 athfa3c3975 altpub cvcv=2 smf=0"
             , "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36 RuxitSynthetic/1.0 v4650801409596051110 t3369586550842097326 ath1fb31b7a altpriv cvcv=2 smf=0"]

def dislikes(videoId, likeCount = None):
    url = "https://returnyoutubedislikeapi.com/votes?videoId=" + videoId

    if likeCount != None:
        url.append("&likeCount=" + str(likeCount))

    headers = {'authority': 'returnyoutubedislikeapi.com'
               , 'cache-control': 'max-age=0'
               , 'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="98", "Microsoft Edge";v="98"'
               , 'sec-ch-ua-mobile': '?0'
               , 'sec-ch-ua-platform': '"Windows"'
               , 'upgrade-insecure-requests': '1'
               , 'user-agent': random.choice(userAgent)
               , 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
               , 'sec-fetch-site': 'none'
               , 'sec-fetch-mode': 'navigate'
               , 'sec-fetch-user': '?1'
               , 'sec-fetch-dest': 'document'
               , 'accept-language': 'ru,en;q=0.9,en-GB;q=0.8,en-US;q=0.7'}

    response = requests.request("GET", url, headers = headers)
    return response.json()["dislikes"]

In [ ]:
## 17.2 собираем через цикл дизлайки
indeseS = video_ne.index
# пустой список errorS для ошибок
errorS = []
for j in indeseS:
    time.sleep(1)
    print(f"Строка №{j}")
    try:
        video_ne.loc[j, "statistics.dislikesCount"] = dislikes(video_ne["id"][j]) # новый столбец "statistics.dislikesCount" в датафрейме, где количество дизлайков у каждого видео
    except:
        print(f"Ошибка в строке {j}")
        errorS.append(j)
video_ne

(возможно, что код будет отрабатывать очень долго в этом чанке, минут 8, поэтому не бейте палками, он очень старается)

Теперь наша задачи состоит в том, чтобы вытащить года видео и посчитать вовлеченность для решения задач. Вовлеченность мы определяем как (лайки+дизлайки+комментарии)/количество просмотров.

In [40]:
# № 18 переменная года выпуска
video_ne["year"] = video_ne["snippet.publishedAt"].apply(lambda yyyy: yyyy[:4])
video_ne["year"] = video_ne["year"].astype("int")

In [41]:
# № 19 работаем с  новой переменной, вовлеченностью в контент (видео)
# ищем и заменяем пропуски, зачастую это связано с тем, что комментарии отключены, то есть их нет у видео,
#  поэтому было принято решение перекодировать их в 0, чтобы не терять наблюдения, тоже самое с лайками
video_ne["statistics.commentCount"] = video_ne["statistics.commentCount"].fillna(0)
video_ne["statistics.likeCount"] = video_ne["statistics.likeCount"].fillna(0)

video_ne["statistics.commentCount"] = video_ne["statistics.commentCount"].astype("int")
video_ne["statistics.likeCount"] = video_ne["statistics.likeCount"].astype("int")
video_ne["statistics.dislikesCount"] = video_ne["statistics.dislikesCount"].astype("int")
video_ne["statistics.viewCount"] = video_ne["statistics.viewCount"].astype("int")

video_ne["involvement"] = round((video_ne["statistics.likeCount"] + video_ne["statistics.dislikesCount"] + video_ne["statistics.commentCount"]) / video_ne["statistics.viewCount"], 2)

In [42]:
video_ne[video_ne["involvement"] == math.inf] #проверили нет ли случайно бесконечности в таблице, ура ее нет

,kind,etag,id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,...,topicDetails.topicCategories,snippet.tags,snippet.defaultLanguage,snippet.defaultAudioLanguage,contentDetails.regionRestriction.blocked,contentDetails.contentRating.ytRating,categoryName,statistics.dislikesCount,year,involvement


In [43]:
# № 20 наконец выбираем только нужные нам категории для будущей готовой таблицы
category = ["id"
             , "snippet.publishedAt"
             , "snippet.channelId"
             , "snippet.title"
             , "snippet.description"
             , "snippet.tags"
             , "snippet.categoryId"
             , "topicDetails.topicCategories"
             , "snippet.channelTitle"
             , "contentDetails.duration"
             , "statistics.viewCount"
             , "statistics.likeCount"
             , "statistics.commentCount"
             , "categoryName"
             , "statistics.dislikesCount"
             , "involvement"
             , "year"]
data_neiro = pandas.DataFrame()

for i in video_ne:
    if i in category:
        data_neiro [i] = video_ne[i]

data_neiro # готовые собранные данные записанные в новую таблицу для дальнейшей работы

,id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.channelTitle,snippet.categoryId,contentDetails.duration,statistics.viewCount,statistics.likeCount,statistics.commentCount,topicDetails.topicCategories,snippet.tags,categoryName,statistics.dislikesCount,year,involvement
1,3OxtnK1m5Sw,2022-10-20T22:44:44Z,UCm7bpbe7_H_47ef4dJH0SNw,Топ-3 нейросети #айти #нейросети,,YCLA-Coding,22,PT36S,11005576,801306,2094,[https://en.wikipedia.org/wiki/Knowledge],NaN,People & Blogs,125318,2022,0.08
2,h6QZtd-Efko,2023-05-04T09:59:13Z,UCf2ZCvUQRwdkFUfebyy0Xag,Лютая сладость от нейросети) Часть 1,,Добрый Джонни,22,PT59S,10420652,717458,1137,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,NaN,People & Blogs,60737,2023,0.07
3,HaLZtHN83JE,2023-01-26T13:00:15Z,UCRd8_F4-SynaG4sE5lnXv_g,Как создать игру если Искусственный Интеллект ...,"Этот туториал поможет вам ответить на вопрос, ...",GameDev Club,24,PT54S,9157840,420422,4183,[https://en.wikipedia.org/wiki/Knowledge],"[как создать игру, как сделать игру, создание ...",Entertainment,143627,2023,0.06
4,gkDrVYInSaQ,2023-02-10T14:00:24Z,UCUEQBzSZx65-FcVwce8HYNQ,Нейросеть побила мировой рекорд! #игры #компью...,,Роман Сакутин - GameDev,27,PT42S,6481887,320103,805,[https://en.wikipedia.org/wiki/Video_game_cult...,NaN,Education,54082,2023,0.06
5,P8GXJ7AWqIA,2022-12-05T07:04:35Z,UCkF05U4kMclJJ8Rp6UzrL_w,Нейросеть превращает изображения в АНИМЕ Vol.2,,amindblow,22,PT21S,6269361,293735,2720,"[https://en.wikipedia.org/wiki/Entertainment, ...",NaN,People & Blogs,31722,2022,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,ZxIN6vCIMe8,2022-06-19T12:36:11Z,UCxMBGX-c9pGG-YOGzdMV1pQ,Нейросеть пояснила за Prey (2006),"В сегодняшней телепрограмме мы узнаем, что же ...",Qwetux 2009,1,PT4M33S,3544,328,66,"[https://en.wikipedia.org/wiki/Action_game, ht...","[нейросеть, искусственный интеллект, ии, машин...",Film & Animation,3,2022,0.11
885,bhsPu9kaMeA,2023-06-04T09:00:20Z,UCSBeL28cCqIyHFxmCTK1Ejw,Обучаем Нейросеть без Программирования,Обучаем искусственный интеллект без программир...,Айтигенио: международная онлайн-школа,20,PT6M23S,3918,39,11,[https://en.wikipedia.org/wiki/Knowledge],"[Айтигенио, itgenio, python, python для детей,...",Gaming,0,2023,0.01
886,WGntxEFMdXo,2023-05-12T12:28:57Z,UCOFQ45F6JIVM1Q7KI-gYT3A,"Господи, как я смеялась 😂 почему меня так не л...",,Анна Булгак,10,PT31S,2667,63,3,[https://en.wikipedia.org/wiki/Entertainment],"[Нейросеть, Kandinsky, Ии, Аннабулгак]",Music,0,2023,0.02
887,j3od_Wn58DQ,2023-04-25T21:10:31Z,UC-zQQEt2z8kO-lyhaJIJmaQ,Новая бесплатная нейросеть Kandinsky 2.1 от Сб...,"Кандинский — обновленная нейросеть, которая ге...",Нейросетевая Лаборатория,28,PT8M24S,2139,104,5,[https://en.wikipedia.org/wiki/Knowledge],"[Нейросеть, Нейронная сеть, Neural network, ис...",Science & Technology,0,2023,0.05


# Ура, пристпупаем к анализу данных!
Мы начали со второй задачи, так как когда-то она показалась нам проще.
#### Задача 2. Выяснить связана ли вовлеченность и год выпуска видео

Для решения этой задачи мы решили прибегнуть к коэфициэнту корреляции Пирсона для нахождения парных связей.

 **Гипотезы:**

 H0: вовлеченность в контент (видео) не связана с годом выпуска видео. Коэффициент corr = 0

 H1: вовлеченность в контент (видео) связана с годом выпуска видео (чем позже опубликовано видео (2023 год), тем выше уровень вовлеченности так, как популярность нейросеть стала проявлять в массовой культуре относительно недавно). Коэффициент corr не равен 0.

In [44]:
# № 22 строим графики, чтобы проанализировать переменные визуально
fig221 = px.histogram(data_neiro
                      , x = "involvement"
                      , template = "simple_white"
                      , title = "Распределение видео по запросу 'нейросеть' ")
fig221.update_layout(yaxis_title = "Частота"
                     , xaxis_title = "Вовлеченность в контент, баллы"
                     , title = {"font": dict(size = 15)
                                , "x": 0.5})
fig221.update_traces(hovertemplate = "Частота: %{y}")

fig221.show()

Можно заметить, что график похож на нормальное смещенное распределение,за исключением выбросов.
Скорее всего в дальнейшем при подсчете корреляции, мы избавимся от них.
Посмотрим внимательнее на выбросы до этого.

In [45]:
fig222 = px.box(data_neiro
                , y = "involvement"
                , points = "all"
                , template = "simple_white"
                , title = "Ящик с усами для переменной 'Вовлеченность в контент (видео)'")
fig222.update_layout(yaxis_title = "Вовлеченность в контент (видео), баллы"
                     , title = {"font": dict(size = 15)
                                , "x": 0.5})

fig222.show()

In [46]:
fig231 = px.histogram(data_neiro
                      , x = "year"
                      , template = "simple_white"
                      , title = "Частотное распределение видео по запросу 'нейросеть' <br>по годам выпуска")
fig231.update_layout(yaxis_title = "Частота"
                     , xaxis_title = "Год выпуска видео"
                     , title = {"font": dict(size = 15)
                               , "x": 0.5})
fig231.update_traces(hovertemplate = "Год: %{x} <br>Частота: %{y}")

fig231.show()

Как мы можем видеть на графике существенная популяризация искусственного интелекта начала происходить в последние два года. Что, возможно, в дальнейшем подтвердит нашу гипотезу. Посмотрим как будут отображаться выбросы

In [48]:
fig232 = px.box(data_neiro
                , y = "year"
                , points = "all"
                , template = "simple_white"
                , title = "Ящик с усами для переменной 'Год выпуска видео'")
fig232.update_layout(yaxis_title = "Год выпуска видео"
                     , title = {"font": dict(size = 15)
                                , "x": 0.5})


fig232.show()


In [49]:
# № 23 посмотрим на статистики/показатели для анализа распределения у "Вовлеченности в контент (видео)" и "Года выпуска"
statistic1 = ScaleStatistics(data_neiro, ["involvement", "year"], normality_test = True)


SCALE STATISTICS SUMMARY
------------------



,N,mode,median,mean,25%,75%,interquart. range,interquart. range (norm.),min,max,range,std,var,entropy coef.,quality var.
involvement,887.000,0.040,0.050,0.061,0.040,0.070,0.030,0.023,0.000,1.330,1.330,0.065,0.004,0.778,0.937
year,888.000,2023.000,2023.000,2022.753,2023.000,2023.000,0.000,0.000,2017.000,2023.000,6.000,0.692,0.479,0.305,0.324



NORMALITY TESTS
------------------



,statistic,p-value
involvement,0.206,0.000
year,0.482,0.000


In [51]:
# № 24 посчитаем корреляцию Пирсона
correlation_pearson_1 = Correlation(data_neiro[["involvement"
                                                  , "year"]])


CORRELATION SUMMARY (PEARSON METHOD, 2-TAILED)
------------------


Note: Each empty index duplicates the previous one.
Maximum correlation is 0.03 (p-value 0.376) for variables involvement and year,
minimum correlation is 0.03 (p-value 0.376) for variables involvement and year.


In [52]:
correlation_pearson_1.sort_correlations() # смотрим уникальные коэффициенты корреляции и значимости (p-value)

Note: Each empty index duplicates the previous one.


,,Coefficient,p-value
involvement,year,0.029748,0.376208


In [54]:
fig253 = px.scatter(data_neiro
                    , y = "involvement"
                    , x = "year"
                    , template = "simple_white"
                    , title = "График рассеивания для переменных 'Вовлеченность в контент (видео)' и 'Год выпуска видео'")
fig253.update_layout(yaxis_title = "Вовлеченность в контент (видео), баллы"
                     , xaxis_title = "Год выпуска видео"
                     , title = {"font": dict(size = 15)
                                , "x": 0.5})
fig253.update_traces(hovertemplate = "Балл вовлеченности: %{y} <br>Год: %{x}")

fig253.show()



**Интерпретация результатов, полученных в Задаче 2**


**Интерпретация коэффициента корреляции Пирсона.**
Исходя из рассчитанного коэффициента Пирсона и p-value, можно сделать вывод, что нет статистически значимой связи между вовлеченностью в контент и годом выпуска видео в генеральной совокупности. Таким образом, нулевая гипотеза подтверждается, а альтернативная гипотеза отвергается.


**Интерпретация графика.**
По графику видно, что, чем позже опубликовано видео, тем выше уровень вовлеченности; можно сказать о наличии линейной связи между переменными.

#### Задача 1. Выяснить, связана ли вовлеченность в контент (видео) с категорией видео


**Для поиска парной связи будет использован** критерий Хи-квадрат Пирсона.


**Гипотезы:**
- H0: вовлеченность в контент (видео) не связана с категорией видео, в генеральной совокупности связи между переменными (признаками) нет.
- H1: вовлеченность в контент (видео) связана с категорией видео, в генеральной совокупности связь между переменными (признаками) есть.



In [57]:
# № 25 строим гистограммы и боксплоты для переменных
fig25 = px.histogram(data_neiro
                    , x = "involvement"
                    , template = "simple_white"
                      , title = "Частотное распределение видео по запросу 'нейросеть' ")

fig25.update_layout(yaxis_title = "Частота"
                     , xaxis_title = "Вовлеченность в контент (видео), баллы"
                     , title = {"font": dict(size = 15)
                                , "x": 0.5})

fig25.update_traces(hovertemplate = "Частота: %{y}")

fig25.show()

In [58]:
fig26 = px.box(data_neiro
                , y = "involvement"
                , points = "all"
                , template = "simple_white"
                , title = "Ящик с усами для переменной 'Вовлеченность в контент (видео)'")


fig26.update_layout(yaxis_title = "Вовлеченность в контент (видео), баллы"
                     , title = {"font": dict(size = 15)
                                , "x": 0.5})

fig26.show()

In [63]:
# №26 подготовка данных для построения столбчатой диаграммы для переменной "Категория видео"
data_fig27 = NominalStatistics(data_neiro, ["categoryName"]).frequencies()['categoryName']


NOMINAL STATISTICS SUMMARY
------------------



,N,mode,entropy coef.,quality var.
categoryName,888,People & Blogs,0.752,0.877



FREQUENCIES
------------------

variable: categoryName


,N,%
People & Blogs,308,34.685
Education,121,13.626
Entertainment,117,13.176
Gaming,90,10.135
Science & Technology,80,9.009
Music,45,5.068
News & Politics,35,3.941
Film & Animation,35,3.941
Howto & Style,21,2.365
Comedy,18,2.027


In [65]:
fig272 = px.bar(data_fig27
                , x = data_fig27.index
                , y = "N"
                , title = "Частотное распределение видео по запросу 'нейросеть' по категориям"
                , template = "simple_white"
                , color = data_fig27.index)

fig272.update_layout(yaxis_title = "Частота"
                     , xaxis_title = ""
                     , title = {"font": dict(size = 15)
                                , "x": 0.5}
                     , showlegend = False)

fig272.update_traces(hovertemplate = "Частота: %{y}")

fig272.show()

In [67]:
# №27 выводим для интервальной переменной "Вовлеченность в контент (видео)" описательные статистики
SS_TASK2 = ScaleStatistics(data_neiro, ["involvement"], normality_test = True)


SCALE STATISTICS SUMMARY
------------------



,N,mode,median,mean,25%,75%,interquart. range,interquart. range (norm.),min,max,range,std,var,entropy coef.,quality var.
involvement,887.000,0.040,0.050,0.061,0.040,0.070,0.030,0.023,0.000,1.330,1.330,0.065,0.004,0.778,0.937



NORMALITY TESTS
------------------



,statistic,p-value
involvement,0.206,0.000


In [69]:
# №28 выводим для номинальной переменной "Категория видео" описательные статистики
NS_TASK2 = NominalStatistics(data_neiro, ["categoryName"])


NOMINAL STATISTICS SUMMARY
------------------



,N,mode,entropy coef.,quality var.
categoryName,888,People & Blogs,0.752,0.877



FREQUENCIES
------------------

variable: categoryName


,N,%
People & Blogs,308,34.685
Education,121,13.626
Entertainment,117,13.176
Gaming,90,10.135
Science & Technology,80,9.009
Music,45,5.068
News & Politics,35,3.941
Film & Animation,35,3.941
Howto & Style,21,2.365
Comedy,18,2.027


In [70]:
# №29 выводим список квартилей переменной "Вовлеченность в контент (видео)"
Involvement_Borders = [round(SS_TASK2.summary()["25%"]["involvement"], 2) # 1 квартиль переменной
                       , round(SS_TASK2.summary()["median"]["involvement"], 2) # 2 квартиль или медианное значение по переменной
                       , round(SS_TASK2.summary()["75%"]["involvement"], 2) # 3 квартиль переменной
                       , round(SS_TASK2.summary()["max"]["involvement"], 2)] # Максимальное значение по переменной
Involvement_Borders

[0.04, 0.05, 0.07, 1.33]

In [72]:
# задаем названия групп/категорий для переменной "Вовлеченность в контент (видео)"
Involvement_Labels = ["1 Till 0.04 Low Involvement"
                      , "2 Till 0.05 Lower Medium Involvement"
                      , "3 Till 0.07 Higher Medium Involvement"
                      , "4 Till 1.13 High Involvement"]
Involvement_Labels

['1 Till 0.04 Low Involvement',
 '2 Till 0.05 Lower Medium Involvement',
 '3 Till 0.07 Higher Medium Involvement',
 '4 Till 1.13 High Involvement']

In [73]:
# записываем название для значений группы/категории с низкой вовлеченностью в контент (видео)
data_neiro.loc[data_neiro["involvement"] <= Involvement_Borders[0], "involvement_level"] = Involvement_Labels[0]
data_neiro[data_neiro["involvement"] <= Involvement_Borders[0]] # Выводим видео только с "1 Till 0.01 Low Involvement" и проверяем, все ли правильно записалось

,id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.channelTitle,snippet.categoryId,contentDetails.duration,statistics.viewCount,statistics.likeCount,statistics.commentCount,topicDetails.topicCategories,snippet.tags,categoryName,statistics.dislikesCount,year,involvement,involvement_level
6,fwR5qlblDzc,2023-05-13T09:43:48Z,UCHoJ7DYrVfKRmD1I8XWjMzA,"ему больше 30🤣нейросеть, которая определит тво...",,FINIK & SMOORFY,22,PT24S,5949877,230875,283,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,NaN,People & Blogs,7078,2023,0.04,1 Till 0.04 Low Involvement
8,7XsdO8J1tII,2022-12-08T07:05:15Z,UCkF05U4kMclJJ8Rp6UzrL_w,Нейросеть скрестила мем и Dota 2,,amindblow,23,PT18S,5620926,119406,1086,"[https://en.wikipedia.org/wiki/Action_game, ht...",NaN,Comedy,25405,2022,0.03,1 Till 0.04 Low Involvement
9,4eFth-C8O2A,2023-04-20T14:15:00Z,UCVMkXkbfVRmjP615zdvHvLA,Президентов нарядили в розовый: новая фантазия...,Ничего милее вы сегодня не увидите. Нейросеть ...,"Короче, новости",25,PT14S,5531638,217689,4555,"[https://en.wikipedia.org/wiki/Politics, https...","[короче, новости, короче новости, кароче, сроч...",News & Politics,10638,2023,0.04,1 Till 0.04 Low Involvement
19,0vbk1wG7gqs,2019-11-10T12:13:50Z,UCnlNy4YSuOcs8Qy6kRoMFbw,НА ЧТО СПОСОБЕН ИСКУССТВЕННЫЙ ИНТЕЛЛЕКТ ОТ OPE...,➥ Подпишись - http://bit.ly/KosmoStory\n➥ Наш ...,Kosmo Story,22,PT4M49S,2482997,94597,2574,[https://en.wikipedia.org/wiki/Puzzle_video_ga...,"[openai, ai, ИСКУССТВЕННЫЙ ИНТЕЛЛЕКТ, Илон мас...",People & Blogs,1406,2019,0.04,1 Till 0.04 Low Involvement
20,0HVIIYrBuKg,2022-10-07T20:34:20Z,UChi1KfffybXf30BcqQn6s-Q,Нейросеть Предсказала чем закончится контрнас...,В сеть слили планы наступления ВСУ в динамике....,Хобби и Развлечения,25,PT2M52S,2352043,11792,24282,"[https://en.wikipedia.org/wiki/Politics, https...","[нейросеть, контрнаступление, всу, предсказани...",News & Politics,29362,2022,0.03,1 Till 0.04 Low Involvement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882,xaPtYETsIIw,2023-02-20T04:00:30Z,UCrnGjnWe_Q7vl8cR9iRXAfQ,"Нейросеть изобразила, как выглядят города Росс...",Почти все портреты получились довольно мрачным...,Интересная планета,22,PT57S,4313,143,7,[https://en.wikipedia.org/wiki/Knowledge],NaN,People & Blogs,0,2023,0.03,1 Till 0.04 Low Involvement
883,LiSqODp1fTo,2021-08-13T04:51:36Z,UCrlnaTh41nSRX5krPJ2Zqfw,"Как найти человека по фото, нейросеть и искусс...","Сейчас XXI век, который по праву считается “Пи...",LaptopWorld - Мир Ноутбуков,28,PT10M26S,4059,76,6,[https://en.wikipedia.org/wiki/Knowledge],"[поиск по фото, как найти человека по фото, ис...",Science & Technology,5,2021,0.02,1 Till 0.04 Low Involvement
885,bhsPu9kaMeA,2023-06-04T09:00:20Z,UCSBeL28cCqIyHFxmCTK1Ejw,Обучаем Нейросеть без Программирования,Обучаем искусственный интеллект без программир...,Айтигенио: международная онлайн-школа,20,PT6M23S,3918,39,11,[https://en.wikipedia.org/wiki/Knowledge],"[Айтигенио, itgenio, python, python для детей,...",Gaming,0,2023,0.01,1 Till 0.04 Low Involvement
886,WGntxEFMdXo,2023-05-12T12:28:57Z,UCOFQ45F6JIVM1Q7KI-gYT3A,"Господи, как я смеялась 😂 почему меня так не л...",,Анна Булгак,10,PT31S,2667,63,3,[https://en.wikipedia.org/wiki/Entertainment],"[Нейросеть, Kandinsky, Ии, Аннабулгак]",Music,0,2023,0.02,1 Till 0.04 Low Involvement


In [74]:
# записываем названия для значений остальных групп/категорий переменной "Уровень вовлеченности в контент (видео)" с помощью цикла for
for i in range(1, len(Involvement_Borders)):
    data_neiro.loc[(data_neiro["involvement"] > Involvement_Borders[i-1])
                  & (data_neiro["involvement"] <= Involvement_Borders[i])
                 , "involvement_level"] = Involvement_Labels[i] # задаем условие для записи оставшихся названий групп/категорий
    print(f"Итерация {i}")

data_neiro

Итерация 1
Итерация 2
Итерация 3


,id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.channelTitle,snippet.categoryId,contentDetails.duration,statistics.viewCount,statistics.likeCount,statistics.commentCount,topicDetails.topicCategories,snippet.tags,categoryName,statistics.dislikesCount,year,involvement,involvement_level
1,3OxtnK1m5Sw,2022-10-20T22:44:44Z,UCm7bpbe7_H_47ef4dJH0SNw,Топ-3 нейросети #айти #нейросети,,YCLA-Coding,22,PT36S,11005576,801306,2094,[https://en.wikipedia.org/wiki/Knowledge],NaN,People & Blogs,125318,2022,0.08,4 Till 1.13 High Involvement
2,h6QZtd-Efko,2023-05-04T09:59:13Z,UCf2ZCvUQRwdkFUfebyy0Xag,Лютая сладость от нейросети) Часть 1,,Добрый Джонни,22,PT59S,10420652,717458,1137,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,NaN,People & Blogs,60737,2023,0.07,3 Till 0.07 Higher Medium Involvement
3,HaLZtHN83JE,2023-01-26T13:00:15Z,UCRd8_F4-SynaG4sE5lnXv_g,Как создать игру если Искусственный Интеллект ...,"Этот туториал поможет вам ответить на вопрос, ...",GameDev Club,24,PT54S,9157840,420422,4183,[https://en.wikipedia.org/wiki/Knowledge],"[как создать игру, как сделать игру, создание ...",Entertainment,143627,2023,0.06,3 Till 0.07 Higher Medium Involvement
4,gkDrVYInSaQ,2023-02-10T14:00:24Z,UCUEQBzSZx65-FcVwce8HYNQ,Нейросеть побила мировой рекорд! #игры #компью...,,Роман Сакутин - GameDev,27,PT42S,6481887,320103,805,[https://en.wikipedia.org/wiki/Video_game_cult...,NaN,Education,54082,2023,0.06,3 Till 0.07 Higher Medium Involvement
5,P8GXJ7AWqIA,2022-12-05T07:04:35Z,UCkF05U4kMclJJ8Rp6UzrL_w,Нейросеть превращает изображения в АНИМЕ Vol.2,,amindblow,22,PT21S,6269361,293735,2720,"[https://en.wikipedia.org/wiki/Entertainment, ...",NaN,People & Blogs,31722,2022,0.05,2 Till 0.05 Lower Medium Involvement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,ZxIN6vCIMe8,2022-06-19T12:36:11Z,UCxMBGX-c9pGG-YOGzdMV1pQ,Нейросеть пояснила за Prey (2006),"В сегодняшней телепрограмме мы узнаем, что же ...",Qwetux 2009,1,PT4M33S,3544,328,66,"[https://en.wikipedia.org/wiki/Action_game, ht...","[нейросеть, искусственный интеллект, ии, машин...",Film & Animation,3,2022,0.11,4 Till 1.13 High Involvement
885,bhsPu9kaMeA,2023-06-04T09:00:20Z,UCSBeL28cCqIyHFxmCTK1Ejw,Обучаем Нейросеть без Программирования,Обучаем искусственный интеллект без программир...,Айтигенио: международная онлайн-школа,20,PT6M23S,3918,39,11,[https://en.wikipedia.org/wiki/Knowledge],"[Айтигенио, itgenio, python, python для детей,...",Gaming,0,2023,0.01,1 Till 0.04 Low Involvement
886,WGntxEFMdXo,2023-05-12T12:28:57Z,UCOFQ45F6JIVM1Q7KI-gYT3A,"Господи, как я смеялась 😂 почему меня так не л...",,Анна Булгак,10,PT31S,2667,63,3,[https://en.wikipedia.org/wiki/Entertainment],"[Нейросеть, Kandinsky, Ии, Аннабулгак]",Music,0,2023,0.02,1 Till 0.04 Low Involvement
887,j3od_Wn58DQ,2023-04-25T21:10:31Z,UC-zQQEt2z8kO-lyhaJIJmaQ,Новая бесплатная нейросеть Kandinsky 2.1 от Сб...,"Кандинский — обновленная нейросеть, которая ге...",Нейросетевая Лаборатория,28,PT8M24S,2139,104,5,[https://en.wikipedia.org/wiki/Knowledge],"[Нейросеть, Нейронная сеть, Neural network, ис...",Science & Technology,0,2023,0.05,2 Till 0.05 Lower Medium Involvement


In [75]:
# №30 вывели таблицу сопреженности
CROSSTAB_TASK2 = Crosstab(data_neiro, "categoryName", "involvement_level")


CROSSTAB SUMMARY
------------------

Observed frequencies


involvement_level,1 Till 0.04 Low Involvement,2 Till 0.05 Lower Medium Involvement,3 Till 0.07 Higher Medium Involvement,4 Till 1.13 High Involvement,Total
categoryName,,,,,
Autos & Vehicles,2,0,0,2,4
Comedy,4,3,4,7,18
Education,46,18,30,26,120
Entertainment,42,19,28,28,117
Film & Animation,13,5,7,10,35
Gaming,14,10,20,46,90
Howto & Style,5,1,6,9,21
Music,20,6,9,10,45
News & Politics,14,12,5,4,35


------------------

Expected frequencies


involvement_level,1 Till 0.04 Low Involvement,2 Till 0.05 Lower Medium Involvement,3 Till 0.07 Higher Medium Involvement,4 Till 1.13 High Involvement,Total
categoryName,,,,,
Autos & Vehicles,1.533,0.609,0.875,0.983,4
Comedy,6.900,2.740,3.937,4.424,18
Education,45.998,18.264,26.246,29.493,120
Entertainment,44.848,17.807,25.590,28.755,117
Film & Animation,13.416,5.327,7.655,8.602,35
Gaming,34.498,13.698,19.684,22.120,90
Howto & Style,8.050,3.196,4.593,5.161,21
Music,17.249,6.849,9.842,11.060,45
News & Politics,13.416,5.327,7.655,8.602,35


25 (41.667%) cells have expected frequency less than 5. The minimum expected frequency is 0.152.
------------------

Chi-square statistic is 84.264 (p-value = 0.0).
------------------

Pearson's residuals


involvement_level,1 Till 0.04 Low Involvement,2 Till 0.05 Lower Medium Involvement,3 Till 0.07 Higher Medium Involvement,4 Till 1.13 High Involvement
categoryName,,,,
Autos & Vehicles,0.377,-0.780,-0.935,1.026
Comedy,-1.104,0.157,0.032,1.225
Education,0.000,-0.062,0.733,-0.643
Entertainment,-0.425,0.283,0.476,-0.141
Film & Animation,-0.114,-0.142,-0.237,0.477
Gaming,-3.490,-0.999,0.071,5.078
Howto & Style,-1.075,-1.228,0.657,1.690
Music,0.662,-0.324,-0.268,-0.319
News & Politics,0.159,2.891,-0.960,-1.569


5 (8.333%) cells have Pearson's residual bigger than 1.96. 
The biggest residual is 5.078 (categories Gaming and 4 Till 1.13 High Involvement).
The smallest residual is -3.49 (categories Gaming and 1 Till 0.04 Low Involvement).


In [76]:
CROSSTAB_TASK2.pvalue #смотрим p-value

0.00011828824913885281

In [77]:
# выводим отдельно положительные значимые остатки критерия Хи-квадрат
CROSSTAB_TASK2.residuals_pearson > 1.96

involvement_level,1 Till 0.04 Low Involvement,2 Till 0.05 Lower Medium Involvement,3 Till 0.07 Higher Medium Involvement,4 Till 1.13 High Involvement
categoryName,,,,
Autos & Vehicles,False,False,False,False
Comedy,False,False,False,False
Education,False,False,False,False
Entertainment,False,False,False,False
Film & Animation,False,False,False,False
Gaming,False,False,False,True
Howto & Style,False,False,False,False
Music,False,False,False,False
News & Politics,False,True,False,False


In [78]:
# тоже самое для отдельно отрицательных значимых остатков критерия Хи-квадрат
CROSSTAB_TASK2.residuals_pearson < -1.96

involvement_level,1 Till 0.04 Low Involvement,2 Till 0.05 Lower Medium Involvement,3 Till 0.07 Higher Medium Involvement,4 Till 1.13 High Involvement
categoryName,,,,
Autos & Vehicles,False,False,False,False
Comedy,False,False,False,False
Education,False,False,False,False
Entertainment,False,False,False,False
Film & Animation,False,False,False,False
Gaming,True,False,False,False
Howto & Style,False,False,False,False
Music,False,False,False,False
News & Politics,False,False,False,False


In [79]:
# сохраняем чтоб наверняка наши данные в excel
data_neiro.to_excel("Проект нейросеть.xlsx")


**Интерпретация результатов для критерия Хи-квадрат.**
На уровне 95% доверительной вероятности нулевая гипотеза об отсутствии связи отклоняется (p-value = 0.0, что меньше, чем 0.05). Следовательно, связь между вовлеченностью в контент (видео) и категорией видео есть. Однако, поскольку процент ненаполненных ячеек превышает допустимое пороговое значение (20%) и составляет 84.264%, доверять полученным результатам критерия Хи-квадрат нельзя. В таком случае мы можем проанализировать и проинтерпретировать статистически значимые остатки критерия Хи-квадрат.

Исходя из рассчитанного значения Хи-квадрат и p-value, можно сделать вывод, что есть статистически значимая связь между вовлеченностью в контент и категорией видео в генеральной совокупности. Таким образом, нулевая гипотеза отвергается, а альтернативная гипотеза подтверждается. Значение Хи-квадрат показывает, что различия между ожидаемыми и фактическими значениями частот значительны. Результаты Pearson's residual показывают, какие ячейки таблицы имеют наибольшее отклонение от ожидаемых значений. Например, категории Gaming и высокая вовлеченность имеют наибольшее положительное отклонение. С другой стороны, категории People & Blogs и низкая вовлеченность имеют наибольшее отрицательное отклонение, что может указывать на то, что видео в этой категории чаще вызывают высокую вовлеченность зрителей. В целом, результаты анализа позволяют сделать вывод о наличии связи между вовлеченностью в контент и категорией видео в генеральной совокупности.